In [1]:
%bash
git clone https://github.com/GoogleCloudPlatform/training-data-analyst
rm -rf training-data-analyst/.git

Cloning into 'training-data-analyst'...


Getting started with tensorflow


In [2]:
import tensorflow as tf
import numpy as np
print(tf.__version__)

1.8.0


/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Adding two tensors using numpy

In [4]:
a= np.array([5,3,2])
b=np.array([6,1,8])
c= np.add(a,b)
print c

[11  4 10]


Equivalent code in TensorFlow has two steps

### Step 1: Build a graph

In [7]:
a= tf.constant([6,1,9])
b= tf.constant([2,3,4])
c= tf.add(a,b)
print c

Tensor("Add_2:0", shape=(3,), dtype=int32)


c is an Op ("Add") that returns a tensor of shape (3,) and holds int32. The shape is inferred from the computation graph.

### Run a Graph

In [13]:
with tf.Session() as sess:
  result = sess.run(c)
  print result
  

[ 8  4 13]


<h2> Using a feed_dict </h2>

Same graph, but without hardcoding inputs at build stage

In [16]:
a= tf.placeholder(dtype=tf.int32, shape=(None,)) #batchsize x scalar
b= tf.placeholder(dtype=tf.int32, shape=(None,))
c= tf.add(a,b) 
with tf.Session() as sess:
  result= sess.run(c, feed_dict={
    a: [1,2,3],
    b: [2,3,4]
  })
  print result

[3 5 7]


<h2> Heron's Formula in TensorFlow </h2>

The area of triangle whose three sides are $(a, b, c)$ is $\sqrt{s(s-a)(s-b)(s-c)}$ where $s=\frac{a+b+c}{2}$ 


In [17]:
def compute_area(sides):
  #slice the input to get the sides
  a = sides[:,0]  # 5.0, 2.3
  b = sides[:,1]  # 3.0, 4.1
  c = sides[:,2]  # 7.1, 4.8
  
  #Heron's formula
  s= (a + b + c) *0.5
  areasq = s * (s - a) * (s - b) * (s - c)
  return tf.sqrt(areasq)

with tf.Session() as sess:
  #pass into two triangles
  area= compute_area(tf.constant([
      [5.0, 3.0, 7.1],
      [2.3, 4.1, 4.8]
    ]))
  
  result= sess.run(area)
  print result

[6.278497 4.709139]


<h2> Placeholder and feed_dict </h2>

More common is to define the input to a program as a placeholder and then to feed in the inputs. The difference between the code below and the code above is whether the "area" graph is coded up with the input values or whether the "area" graph is coded up with a placeholder through which inputs will be passed in at run-time.

In [18]:
with tf.Session() as sess:
  sides= tf.placeholder(tf.float32, shape=(None,3)) #batchsize number of triangles, 3 sides
  area= compute_area(sides)
  result= sess.run(area, feed_dict={
    sides: [
        [5.0, 3.0, 7.1],
        [2.3, 4.1, 4.8]
      ]
  })
  print result

[6.278497 4.709139]


## tf.eager

tf.eager allows you to avoid the build-then-run stages. However, most production code will follow the lazy evaluation paradigm because the lazy evaluation paradigm is what allows for multi-device support and distribution. 
<p>

You may need to click on Reset Session to try this out

In [1]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe

tf.enable_eager_execution()

def compute_area(sides):
  # slice the input to get the sides
  a = sides[:,0]  # 5.0, 2.3
  b = sides[:,1]  # 3.0, 4.1
  c = sides[:,2]  # 7.1, 4.8
  
  # Heron's formula
  s = (a + b + c) * 0.5   # (a + b) is a short-cut to tf.add(a, b)
  areasq = s * (s - a) * (s - b) * (s - c) # (a * b) is a short-cut to tf.multiply(a, b), not tf.matmul(a, b)
  return tf.sqrt(areasq)

area = compute_area(tf.constant([
      [5.0, 3.0, 7.1],
      [2.3, 4.1, 4.8]
    ]))


print area

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


tf.Tensor([6.278497 4.709139], shape=(2,), dtype=float32)
